In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne

from mne_connectivity import (spectral_connectivity_epochs,
                              spectral_connectivity_time)

import warnings
warnings.simplefilter('ignore')

In [3]:
from model_functions import *

In [6]:
from connectivity_plus_rbp_features import *

# Generate connectivity: long epochs:

In [7]:
# 'con_data_01.npy' parameters: Other cases follow similarly:
n_times = 500*120; overlap_ratio = 0.6;
channels = [6,7,8,9]; n_channels = len(channels);
sfreq = 256; f_spacing = 4;
# Freq bands of interest
Freq_Bands = {"delta": [0.5, 4.0],
              "theta": [4.0, 8.0],
              "alpha": [8.0, 13.0],
              "beta": [13.0, 30.0]}
f_bands = ["delta"]; n_freq_bands = len(f_bands)
method = 'plv'; mode = 'cwt_morlet';

In [ ]:
"""
To generate data used for METHOD 2 of "AD-FTD analysis using different methods.ipynb":
1) P3,P4,O1,O2 nodes for delta band: channels = [6,7,8,9]: filename: 'con_data_01.npy'
2) C3,C4,F3,F4 nodes for theta band: channels = [2,3,4,5]: filename: 'con_data_01.npy'
3) C3,C4,F3,F4 nodes for delta band: channels = [2,3,4,5]: filename: 'con_data_01.npy'
4) C3,C4,T3,T4 nodes for theta band: channels = [4,5,12,13]: filename: 'con_data_01.npy'
5) C3,C4,P3,P4 nodes for theta band: channels = [4,5,6,7]: filename: 'con_data_01.npy'

conn_data_1 = np.load('con_data_01.npy',allow_pickle=True)
conn_data_2 = np.load('con_data_02.npy',allow_pickle=True)
conn_data_3 = np.load('con_data_03.npy',allow_pickle=True)
conn_data_4 = np.load('con_data_04.npy',allow_pickle=True)
conn_data_5 = np.load('con_data_05.npy',allow_pickle=True)
"""

In [ ]:
# generate epoched raw eeg data:
all_data = generate_and_save_data(True,n_times,overlap_ratio,save = True, filename = path+ "processed_data/eeg_data_long_epochs.npy")

In [5]:
# spectral connectivity data for all subjects: single choice of parameters:
filename = path + "processed_data/con_data_0"+"INSERT_INTEGER"+".npy";
con_data = connectivity_processing(all_data, channels, Freq_Bands, f_bands, f_spacing, sfreq, method, mode, subjects = [i for i in range(88)], f_avg = True, save = True, filename = filename, ch_types = "eeg")

In [9]:
# uncomment if previously generated data:
#con_data = np.load(filename,allow_pickle=True)

# Generate rbp data: long epochs

In [18]:
# create dictionary of channel names:
channel_dict = {0: 'Fp1', 1: 'Fp2', 2: 'F3', 3: 'F4', 4: 'C3', 5: 'C4', 6: 'P3',
                    7: 'P4', 8: 'O1', 9: 'O2',10: 'F7', 11: 'F8', 12: 'T3', 13: 'T4',
                    14: 'T5', 15: 'T6', 16: 'Fz', 17: 'Cz', 18: 'Pz'}

In [19]:
# reverse dictionary mapping:
probe_dict = dict((v, k) for k, v in channel_dict.items())

In [20]:
# choose parameters:
epoch_length = 60000
overlap_ratio = 0.6
freq_bands = np.array([0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0])
nperseg = 256

# remove channels: 
remove_channel = ['F3', 'F4', 'C4', 'F8'];
channels = [element for element in list(channel_dict) if element not in [probe_dict[j] for j in remove_channel]]

# subset of frequencies:
freqs = [i for i in range(6)]

In [31]:
# generate rbp features:
rbp_filename = path+ "processed_data/rbp_data_long_epochs.npy";
rbp_features = process(epoch_length,overlap_ratio,freq_bands,nperseg, absolute=False, sample_freq=256,total_subjects=88)

In [32]:
np.save(rbp_filename, np.array((np.array(rbp_features, dtype=object)), dtype=object), allow_pickle=True)

In [10]:
# load data if previously generated:
#rbp_data = np.load(rbp_filename,allow_pickle=True)

# Format data for training

# Load pre-generated data

## connectivity data processing:

**NOTE:** Must have several .npy connectivity data files pre-generated that you compile into a list:

In [12]:
# aggregates previously generated connectivity data from various node/frequency combinations:
con_data_lst = [np.load('processed_data/con_data_0'+str(i+1)+'.npy',allow_pickle=True) for i in range(5)]

In [13]:
con_features = get_con_data(con_data_lst,88)

## rbp data processing:

In [14]:
rbp_data = np.load('processed_data/rbp_data_long_epochs.npy',allow_pickle=True)

In [21]:
rbp_features = reshape_rbp_data(rbp_data,freqs,channels)

## merging

In [22]:
features = get_con_rbp_data(con_features,rbp_features,88)

In [25]:
np.save("processed_data/rbp_plus_all_con_long_epochs.npy", np.array((np.array(features, dtype=object)), dtype=object), allow_pickle=True)